In [ ]:
# GPU Connection info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)

if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

In [ ]:
# Print whether using a high VRAM env from google
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
    print('Not using a high-RAM runtime')
else:
    print('You are using a high-RAM runtime!')

In [ ]:
# connect to drive and make necessary imports
from google.colab import drive
from IPython.display import clear_output
import os
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import sys
import fileinput
import os
import time
import base64
import gdown
from gdown.download import get_url_from_gdrive_confirmation
import requests
from urllib.request import urlopen, Request
from urllib.parse import urlparse, parse_qs, unquote
from tqdm import tqdm
import six

print("Connecting...")
drive.mount('/content/gdrive')

In [ ]:
# install or update the A1111 repo

if not os.path.exists('/content/gdrive'):
    print('please connect to google drive before proceeding')


with capture.capture_output() as cap:
    fgitclone = "git clone --depth 1"
    %mkdir -p /content/gdrive/MyDrive/sd
    %cd /content/gdrive/MyDrive/sd
    !git clone -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui
    !mkdir -p /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/
    os.environ['TRANSFORMERS_CACHE'] = f"/content/gdrive/MyDrive/sd/stable-diffusion-webui/cache"
    os.environ['TORCH_HOME'] = f"/content/gdrive/MyDrive/sd/stable-diffusion-webui/cache"
    !mkdir -p /content/gdrive/MyDrive/sd/stable-diffusion-webui/repositories
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-assets /content/gdrive/MyDrive/sd/stable-diffusion-webui/repositories/stable-diffusion-webui-assets

with capture.capture_output() as cap:
    %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/
    !git reset --hard
    !git checkout master
    time.sleep(1)
    !rm webui.sh
    !git pull
print('Completed A1111 Repo Download/Update!')

In [ ]:
# Requirements

print('Installing requirements...')

with capture.capture_output() as cap:
    %cd /content/
    !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
    !dpkg -i *.deb
    if not os.path.exists('/content/gdrive/MyDrive/sd/stablediffusion'):
        !tar -C /content/gdrive/MyDrive --zstd -xf sd_mrep.tar.zst
    !tar -C / --zstd -xf gcolabdeps.tar.zst
    !rm *.deb | rm *.zst | rm *.txt
    !pip install spandrel==0.1.6 --qq
    if not os.path.exists('gdrive/MyDrive/sd/libtcmalloc/libtcmalloc_minimal.so.4'):
        %env CXXFLAGS=-std=c++14
        !wget -q https://github.com/gperftools/gperftools/releases/download/gperftools-2.5/gperftools-2.5.tar.gz && tar zxf gperftools-2.5.tar.gz && mv gperftools-2.5 gperftools
        !wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/Patch
        %cd /content/gperftools
        !patch -p1 < /content/Patch
        !./configure --enable-minimal --enable-libunwind --enable-frame-pointers --enable-dynamic-sized-delete-support --enable-sized-delete --enable-emergency-malloc; make -j4
        !mkdir -p /content/gdrive/MyDrive/sd/libtcmalloc && cp .libs/libtcmalloc*.so* /content/gdrive/MyDrive/sd/libtcmalloc
        %env LD_PRELOAD=/content/gdrive/MyDrive/sd/libtcmalloc/libtcmalloc_minimal.so.4
        %cd /content
        !rm *.tar.gz Patch && rm -r /content/gperftools
    else:
        %env LD_PRELOAD=/content/gdrive/MyDrive/sd/libtcmalloc/libtcmalloc_minimal.so.4

    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    os.environ['PYTHONWARNINGS'] = 'ignore'
    !sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.10/warnings.py

print('Completed requirements installation!')

In [ ]:
# ========================================================================================================================================================================================================================

In [ ]:
# Model Download/Load
# Download Stable Diffusion 1.5 as the base model if it does not exist

def download_sd1_5():
    version = '1.5'
    model = '/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned-emaonly.safetensors'
    link = 'https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors'

    if not os.path.exists(model):
        !gdown --fuzzy -O $model $link
        if os.path.exists(model):
            print('Completed!')
        else:
            print('ERROR! Something went wrong.')
    else:
        print('Model already exists!')

    return model


model = download_sd1_5()

In [ ]:
# ========================================================================================================================================================================================================================

In [ ]:
# Install ControlNet - Only needs to be done once

from torch.hub import download_url_to_file
from urllib.parse import urlparse
import re
from subprocess import run


# potential choices = ["None", "All (21GB)", "Canny", "Depth", "Lineart", "MLSD", "Normal", "OpenPose", "Scribble", "Seg", "ip2p", "Shuffle", "Inpaint", "Softedge", "Lineart_Anime", "Tile", "T2iadapter_Models"]
v1_Model = 'Scribble'


# Download/update ControlNet extension and its models
def download(url, model_dir):
    filename = os.path.basename(urlparse(url).path)
    pth = os.path.abspath(os.path.join(model_dir, filename))
    if not os.path.exists(pth):
        print('Downloading: ' + os.path.basename(url))
        download_url_to_file(url, pth, hash_prefix = None, progress = True)
    else:
        print(f"The model {filename} already exists")


with capture.capture_output() as cap:
    %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions
    if not os.path.exists('sd-webui-controlnet'):
        !git clone https://github.com/Mikubill/sd-webui-controlnet.git
        %cd /content
    else:
        %cd sd-webui-controlnet
        !git reset --hard
        !git pull
        %cd /content

mdldir = '/content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet/models'
for filename in os.listdir(mdldir):
    if "_sd14v1" in filename:
        renamed = re.sub("_sd14v1", "-fp16", filename)
        os.rename(os.path.join(mdldir, filename), os.path.join(mdldir, renamed))

!wget -q -O CN_models.txt https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/CN_models.txt


with open("CN_models.txt", 'r') as f:
    mdllnk = f.read().splitlines()

!rm CN_models.txt


# ControlNet model Links
Canny='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth'
Depth='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth'
Lineart='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth'
MLSD='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth'
Normal='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth'
OpenPose='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth'
Scribble='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth'
Seg='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth'
ip2p='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth'
Shuffle='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth'
Inpaint='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth'
Softedge='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth'
Lineart_Anime='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth'
Tile='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth'


if v1_Model == "All (21GB)":
    for lnk in mdllnk:
        download(lnk, mdldir)

elif v1_Model == "T2iadapter_Models":
    mdllnk = list(filter(lambda x: 't2i' in x, mdllnk))
    for lnk in mdllnk:
        download(lnk, mdldir)

elif v1_Model == "None":
    pass
    print('Nothing to download.')

else:
    download(globals()[v1_Model], mdldir)
    print('Completed downloading ControlNet successfully!')

In [ ]:
# ========================================================================================================================================================================================================================

In [ ]:
# Start Stable Diffusion

from IPython.utils import capture
import time
import sys
import fileinput
import re


with capture.capture_output() as cap:
    %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/
    !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
    %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/

    !sed -i 's@shared.opts.data\["sd_model_checkpoint"] = checkpoint_info.title@shared.opts.data\["sd_model_checkpoint"] = checkpoint_info.title;model.half()@' /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/sd_models.py
    !sed -i "s@map_location='cpu'@map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/extras.py

    !sed -i 's@possible_sd_paths =.*@possible_sd_paths = [\"/content/gdrive/MyDrive/sd/stablediffusion\"]@' /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/paths.py
    !sed -i 's@\.\.\/@src/@g' /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/paths.py
    !sed -i 's@src/generative-models@generative-models@g' /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/paths.py

    !sed -i 's@print(\"No module.*@@' /content/gdrive/MyDrive/sd/stablediffusion/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers", "inpainting_mask_weight", "initial_noise_multiplier"\]@g' /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/shared.py


ckptdir = ''

try:
    model
    if os.path.isfile(model):
        !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py --share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt "$model" --opt-sdp-attention --disable-console-progressbars --skip-version-check $ckptdir
    else:
        !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py --share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt-dir "$model" --opt-sdp-attention --disable-console-progressbars --skip-version-check
except:
    !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py --share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --opt-sdp-attention --disable-console-progressbars --skip-version-check $ckptdir